In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader, Subset
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import random

# --- Reproducibility ---
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# --- Settings ---
images_path = "../output/intersectionlight_images_merged.npy"  # shape: (N, H, W, 3)
labels_path = "../output/intersectionlight_labels_merged.npy"  # shape: (N,) values: 0=red, 1=green

val_size = 0.2
batch_size = 32
epochs = 10
img_size = 224  # ResNet-18 default
lr = 1e-3
num_workers = 2  # tweak per machine

# --- Dataset ---
class NpyDataset(Dataset):
    def __init__(self, images_path, labels_path, transform=None):
        self.images = np.load(images_path)       # (N, H, W, 3), uint8 or float
        self.labels = np.load(labels_path).astype(np.int64)  # (N,)
        assert len(self.images) == len(self.labels), "Images and labels must have same length."
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        lbl = self.labels[idx]
        if self.transform:
            img = self.transform(img)  # expects HWC
        return img, lbl

# --- Transform ---
transform = transforms.Compose([
    transforms.ToPILImage(),  # works with HWC numpy
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# --- Build base dataset and stratified split ---
base_ds = NpyDataset(images_path, labels_path, transform)

sss = StratifiedShuffleSplit(n_splits=1, test_size=val_size, random_state=seed)
(train_idx, val_idx), = sss.split(np.zeros(len(base_ds)), base_ds.labels)

train_ds = Subset(base_ds, train_idx)
val_ds   = Subset(base_ds, val_idx)

# --- DataLoaders ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pin = (device.type == "cuda")
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,
                          num_workers=num_workers, pin_memory=pin)
val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False,
                          num_workers=num_workers, pin_memory=pin)

# --- Model ---
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, 2)  # 2 classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# --- Training ---
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            preds = model(imgs).argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / len(train_ds)
    val_acc = correct / total if total > 0 else 0.0
    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {epoch_loss:.4f} | Val Acc: {val_acc:.4f}")

# Save model
torch.save(model.state_dict(), "traffic_light_resnet18.pth")
